In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import cv2

In [ ]:
from datasets import load_dataset

dataset = load_dataset("jxie/flickr8k")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/6000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
train_data = dataset["train"]
validation_data = dataset["validation"]
test_data = dataset["test"]

In [ ]:
# train_data = pd.read_parquet('/content/drive/My Drive/pattern_recognition/lab_2/train.parquet')
# validation_data = pd.read_parquet('/content/drive/My Drive/pattern_recognition/lab_2/validation.parquet')
# test_data = pd.read_parquet('/content/drive/My Drive/pattern_recognition/lab_2/test.parquet')

#Step 1: Dataset Preparation

##Loading and preparing Images

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input

# Load pre-trained ResNet50 model to know required input dimensions(shape)
resnet_model = ResNet50(weights='imagenet')

# Get input shape of the model
input_shape = resnet_model.input_shape[1:]  # Exclude batch size
print("Input Shape:", input_shape)

102967424/102967424 [==============================] - 1s 0us/step
Input Shape: (224, 224, 3)


###Normal images

In [ ]:
training_images = []
validation_images = []
test_images = []

####Preparing training images

In [ ]:
from PIL import Image
import io
import numpy as np

for i in range(len(train_data)):
  #image_bytes = train_data['image'][i]['bytes']

  # Convert bytes to PIL image
  #image = Image.open(io.BytesIO(image_bytes))

  image = train_data[i]["image"]

  # Resize image to required ResNet specifications
  width, height = 224, 224
  image = np.array(image)
  image = cv2.resize(image, (width, height))
  #image = image.resize((width, height))

  # Normalizes image according to the specification of the ResNet model.
  # This typically involves scaling pixel values to the range [-1, 1] or [0, 1]
  # and performing channel-wise mean normalization.
  #image_array = preprocess_input(image_array)

  #train_data[i]['image'] = image
  training_images.append(image)

In [ ]:
print(training_images[0].shape)

(224, 224, 3)


####Preparing validation images

In [ ]:
from PIL import Image
import io
import numpy as np

for i in range(len(validation_data)):
  #image_bytes = validation_data['image'][i]['bytes']


  # Convert bytes to PIL image
  #image = Image.open(io.BytesIO(image_bytes))

  image = validation_data[i]["image"]

  # Resize image to required ResNet specifications
  width, height = 224, 224
  image = np.array(image)
  image = cv2.resize(image, (width, height))
  #image = image.resize((width, height))

  # Normalizes image according to the specification of the ResNet model.
  # This typically involves scaling pixel values to the range [-1, 1] or [0, 1]
  # and performing channel-wise mean normalization.
  #image_array = preprocess_input(image_array)

  validation_images.append(image)

In [ ]:
print(validation_images[0].shape)

(224, 224, 3)


####Preparing test images

In [ ]:
from PIL import Image
import io
import numpy as np

for i in range(len(validation_data)):
  #image_bytes = validation_data['image'][i]['bytes']


  # Convert bytes to PIL image
  #image = Image.open(io.BytesIO(image_bytes))

  image = test_data[i]["image"]

  # Resize image to required ResNet specifications
  width, height = 224, 224
  image = np.array(image)
  image = cv2.resize(image, (width, height))
  #image = image.resize((width, height))

  # Normalizes image according to the specification of the ResNet model.
  # This typically involves scaling pixel values to the range [-1, 1] or [0, 1]
  # and performing channel-wise mean normalization.
  #image_array = preprocess_input(image_array)

  test_images.append(image)

In [ ]:
print(test_images[0].shape)

(224, 224, 3)


In [ ]:
len(validatio_images)

6001

###Augmented images

In [ ]:
augmented_training_images_1 = []

for image in training_images:
    flipped_image = image[:, ::-1]
    augmented_training_images_1.append(flipped_image)

augmented_training_images_1 = np.array(augmented_training_images_1)

In [ ]:
import os

file_path = '/content/drive/My Drive/pattern_recognition/lab_2/augmented_training_images_1.npy'
np.save(file_path, augmented_training_images_1)
if os.path.exists(file_path):
    print("Augmented training images 1 saved successfully!")
else:
    print("Error: Augmented training images 1 not found.")

Augmented training images 1 saved successfully!


##Preprocessing Captions for RNN

In [ ]:
# Loading training, validation, and test captions
training_captions = []
validation_captions = []
test_captions = []

for i in range(len(train_data)):
  training_captions.append(train_data[i]['caption_0'])

for i in range(len(validation_data)):
  validation_captions.append(validation_data[i]['caption_0'])

for i in range(len(test_data)):
  test_captions.append(test_data[i]['caption_0'])

###Creating vocabulary

In [ ]:
# Choosing a single caption for every image
word_count = {}
vocabulary = {}

# Counting frequency of each word throughout caption list
for caption in training_captions:
  words = caption.split()
  for word in words:
    lowercase_word = word.lower()

    if lowercase_word not in word_count:
      word_count[lowercase_word] = 1
    else:
      word_count[lowercase_word] += 1

# Sorting word_count map by frequency of each word
word_count = sorted(word_count.items(), key=lambda x: x[1], reverse=True)

# indexing word according to their arrangement
index = 1  # start indexing from 1, leave 0 for padding

for word, word_count in word_count:
  vocabulary[word] = index
  index += 1

In [ ]:
vocabulary['startseq'] = 2728
vocabulary['endseq'] = 2729
vocabulary['unk'] = 2730

print(vocabulary)
inv_vocab={v:k for k,v in vocabulary.items()}
print(inv_vocab)

{'a': 1, '.': 2, 'in': 3, 'the': 4, 'and': 5, 'on': 6, 'is': 7, 'dog': 8, 'of': 9, 'man': 10, 'with': 11, 'black': 12, 'boy': 13, 'girl': 14, 'white': 15, 'brown': 16, 'are': 17, 'two': 18, 'at': 19, 'his': 20, 'to': 21, ',': 22, 'an': 23, 'woman': 24, 'red': 25, 'wearing': 26, 'water': 27, 'blue': 28, 'group': 29, 'while': 30, 'people': 31, 'shirt': 32, 'running': 33, 'down': 34, 'playing': 35, 'through': 36, 'standing': 37, 'ball': 38, 'jumping': 39, 'another': 40, 'holding': 41, 'dogs': 42, 'snow': 43, 'child': 44, 'grass': 45, 'yellow': 46, 'over': 47, 'little': 48, 'front': 49, 'by': 50, 'green': 51, 'from': 52, 'field': 53, 'sitting': 54, 'walking': 55, 'pink': 56, 'her': 57, 'mouth': 58, 'for': 59, 'up': 60, 'children': 61, 'runs': 62, 'beach': 63, 'jumps': 64, 'large': 65, 'person': 66, 'its': 67, 'into': 68, 'air': 69, 'jacket': 70, 'as': 71, 'player': 72, 'street': 73, 'riding': 74, 'behind': 75, 'near': 76, 'camera': 77, 'dressed': 78, 'stands': 79, 'skateboard': 80, 'girls'

###Tokenizing captions for original images

####Training captions

In [ ]:
# Getting max caption length
max_len = 0

for i in range(0,len(training_captions)):
  training_captions[i]='startseq '+training_captions[i]+' endseq'

for caption in training_captions:
  caption_length = len(caption.split())

  if caption_length > max_len:
    max_len = caption_length

# Tokenizing captions
training_tokenized_captions = []

for caption in training_captions:
  tokenized_caption = []
  words = caption.split()

  for word in words:
    tokenized_caption.append(vocabulary[word.lower()])

  # padding for shorter captions with 0
  if len(words) < max_len:
    for i in range(max_len - len(words)):
      tokenized_caption.append(0)

  training_tokenized_captions.append(tokenized_caption)

In [ ]:
print(training_tokenized_captions)

[[2728, 1, 12, 8, 7, 33, 262, 1, 15, 8, 3, 4, 43, 2, 2729, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2728, 1, 48, 109, 122, 991, 2, 2729, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2728, 1, 16, 8, 3, 4, 43, 117, 165, 646, 56, 3, 67, 58, 2, 2729, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2728, 1, 16, 8, 7, 33, 100, 1, 63, 2, 2729, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2728, 1, 12, 5, 15, 8, 11, 1, 25, 221, 37, 6, 1, 401, 63, 2, 2729, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2728, 1, 402, 26, 1, 25, 189, 7, 74, 6, 4, 647, 2, 2729, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2728, 1, 10, 78, 3, 1, 198, 32, 5, 25, 403, 263, 19, 4, 31, 199, 130, 2, 2729, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2728, 1, 13, 26, 1, 25, 315, 7, 33, 36, 776, 2, 2729, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

####Validation captions

In [ ]:
# Getting max caption length
max_len = 0

for i in range(0,len(validation_captions)):
  validation_captions[i]='startseq '+validation_captions[i]+' endseq'

for caption in validation_captions:
  caption_length = len(caption.split())

  if caption_length > max_len:
    max_len = caption_length

# Tokenizing captions
validation_tokenized_captions = []

for caption in validation_captions:
  tokenized_caption = []
  words = caption.split()

  for word in words:
    if word.lower() in vocabulary:
      tokenized_caption.append(vocabulary[word.lower()])
    else:
      tokenized_caption.append(vocabulary['unk'])

  # padding for shorter captions with 0
  if len(words) < max_len:
    for i in range(max_len - len(words)):
      tokenized_caption.append(0)

  validation_tokenized_captions.append(tokenized_caption)

In [ ]:
print(validation_tokenized_captions)

[[2728, 4, 13, 214, 139, 34, 6, 1, 80, 7, 143, 2730, 100, 4, 162, 50, 40, 13, 2, 2729, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2728, 1, 13, 3, 1, 28, 121, 7, 39, 89, 90, 258, 3, 4, 339, 2, 2729, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2728, 1, 147, 55, 57, 8, 36, 23, 442, 355, 22, 30, 93, 31, 17, 3, 4, 134, 2, 2729, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2728, 1, 107, 12, 8, 64, 3, 4, 69, 21, 247, 4, 119, 38, 3, 20, 58, 2, 2729, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2728, 18, 24, 106, 258, 101, 4, 151, 2729, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2728, 1, 44, 7, 214, 3, 1, 520, 2249, 41, 1, 46, 2730, 2730, 60, 21, 20, 58, 71, 2509, 2730, 68, 1, 1221, 2, 2729, 0, 0, 0, 0, 0, 0, 0, 0], [2728, 1, 29, 9, 31, 17, 269, 419, 3, 1, 107, 172, 2, 2729, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2728, 170, 61, 17, 35, 3, 1, 149, 53, 2, 2729, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],

####Test captions

In [ ]:
# Getting max caption length
max_len = 0

for i in range(0,len(test_captions)):
  test_captions[i]='startseq '+test_captions[i]+' endseq'

for caption in test_captions:
  caption_length = len(caption.split())

  if caption_length > max_len:
    max_len = caption_length

# Tokenizing captions
test_tokenized_captions = []

for caption in test_captions:
  tokenized_caption = []
  words = caption.split()

  for word in words:
    if word.lower() in vocabulary:
      tokenized_caption.append(vocabulary[word.lower()])
    else:
      tokenized_caption.append(vocabulary['unk'])

  # padding for shorter captions with 0
  if len(words) < max_len:
    for i in range(max_len - len(words)):
      tokenized_caption.append(0)

  test_tokenized_captions.append(tokenized_caption)

In [ ]:
print(test_tokenized_captions)

[[2728, 2728, 4, 42, 17, 3, 4, 43, 3, 49, 9, 1, 166, 2, 2729, 2729, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2728, 2728, 1, 16, 5, 15, 8, 137, 236, 90, 3, 4, 87, 2729, 2729, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2728, 2728, 1, 10, 5, 1, 24, 3, 2730, 687, 335, 2, 2729, 2729, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2728, 2728, 1, 118, 9, 31, 201, 471, 19, 1, 196, 11, 23, 386, 5, 601, 2, 2729, 2729, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2728, 2728, 1, 10, 7, 26, 1, 1478, 25, 113, 32, 5, 189, 2, 2729, 2729, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2728, 2728, 1, 16, 8, 33, 2729, 2729, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2728, 2728, 1, 14, 11, 154, 16, 211, 5, 472, 3, 1, 28, 574, 7, 37, 82, 21, 1, 14, 3, 1, 1637, 2730, 330, 2, 2729, 2729, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2728, 2728, 1, 8, 11, 67, 

###Tokenizing captions for augmented images

In [ ]:
augmented_training_captions = []

for i in range(len(train_data['caption_1'])):
  augmented_training_captions.append(train_data['caption_1'][i])

In [ ]:
# Getting max caption length
max_len = 37

for i in range(0,len(augmented_training_captions)):
  augmented_training_captions[i]='startseq '+augmented_training_captions[i]+' endseq'

for caption in augmented_training_captions:
  caption_length = len(caption.split())

  if caption_length > max_len:
    max_len = caption_length

# Tokenizing captions
augmented_tokenized_captions = []

for caption in augmented_training_captions:
  tokenized_caption = []
  words = caption.split()

  for word in words:
    if word.lower() in vocabulary:
      tokenized_caption.append(vocabulary[word.lower()])
    else:
      tokenized_caption.append(3)

  # padding for shorter captions with 0
  if len(words) < max_len:
    for i in range(max_len - len(words)):
      tokenized_caption.append(0)

  augmented_tokenized_captions.append(tokenized_caption)

In [ ]:
print(augmented_tokenized_captions)

[[2728, 12, 8, 216, 16, 8, 36, 43, 2729, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2728, 1, 48, 14, 122, 991, 82, 21, 1, 746, 2, 2729, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2728, 1, 16, 8, 3, 4, 43, 41, 1, 56, 96, 2, 2729, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2728, 1, 16, 8, 26, 1, 12, 427, 33, 129, 4, 63, 2, 2729, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2728, 1, 8, 3, 1, 25, 2046, 6, 1, 63, 2, 2729, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2728, 1, 14, 7, 74, 1, 86, 6, 4, 73, 30, 26, 1, 25, 189, 2, 2729, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2728, 1, 10, 6, 4, 73, 26, 810, 3, 5, 1, 3, 3, 2, 2729, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2728, 1, 44, 62, 76, 90, 177, 2, 2729, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

##Preparing Output Labels

###Training labels

In [ ]:
# For each sequence, output sequence will be shifted by one
# For example: ["I", "love", "to", "eat"],  corresponding output labels are ["love", "to", "eat", "ice", "cream"].

training_output_labels = []
for tokenized_caption in training_tokenized_captions:
  label = tokenized_caption[1:]
  label.append(0)
  training_output_labels.append(label)

In [ ]:
print(training_output_labels[0])

[1, 12, 8, 7, 33, 262, 1, 15, 8, 3, 4, 43, 2, 2729, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


###Validation labels

In [ ]:
# For each sequence, output sequence will be shifted by one
# For example: ["I", "love", "to", "eat"],  corresponding output labels are ["love", "to", "eat", "ice", "cream"].

validation_output_labels = []
for tokenized_caption in validation_tokenized_captions:
  label = tokenized_caption[1:]
  label.append(0)
  validation_output_labels.append(label)

In [ ]:
print(validation_output_labels[0])

[4, 13, 214, 139, 34, 6, 1, 80, 7, 143, 2730, 100, 4, 162, 50, 40, 13, 2, 2729, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


###Test labels

In [ ]:
# For each sequence, output sequence will be shifted by one
# For example: ["I", "love", "to", "eat"],  corresponding output labels are ["love", "to", "eat", "ice", "cream"].

test_output_labels = []
for tokenized_caption in test_tokenized_captions:
  label = tokenized_caption[1:]
  label.append(0)
  test_output_labels.append(label)

In [ ]:
print(test_output_labels[0])

[4, 42, 17, 3, 4, 43, 3, 49, 9, 1, 166, 2, 2729, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


##Saving vocabulary, tokenized captions, and output labels

In [ ]:
test_tokenized_captions_as_npy = np.array(test_tokenized_captions)

In [ ]:
# Converting vocabulary map to npy to save on Google drive
keys_array = np.array(list(vocabulary.keys()))
values_array = np.array(list(vocabulary.values()))
vocabulary_as_npy = np.column_stack((keys_array, values_array))

# Converting tokenized_captions list to npy to save on Google drive
training_tokenized_captions_as_npy = np.array(training_tokenized_captions)
validation_tokenized_captions_as_npy = np.array(validation_tokenized_captions)
test_tokenized_captions_as_npy = np.array(test_tokenized_captions)
augmented_tokenized_captions_as_npy = np.array(augmented_tokenized_captions)

# Converting output_labels list to npy to save on Google drive
training_output_labels_as_npy = np.array(training_output_labels)
validation_output_labels_as_npy = np.array(validation_output_labels)
test_output_labels_as_npy = np.array(test_output_labels)

In [ ]:
print(vocabulary_as_npy)

[['a' '1']
 ['.' '2']
 ['in' '3']
 ...
 ['startseq' '2728']
 ['endseq' '2729']
 ['unk' '2730']]


In [ ]:
# Saving data to drive
import os

file_path = '/content/drive/My Drive/pattern_recognition/lab_2/vocabulary.npy'
np.save(file_path, vocabulary_as_npy)
if os.path.exists(file_path):
    print("Vocabulary saved successfully!")
else:
    print("Error: Vocabulary not found.")

file_path = '/content/drive/My Drive/pattern_recognition/lab_2/training_tokenized_captions.npy'
np.save(file_path, training_tokenized_captions_as_npy)
if os.path.exists(file_path):
    print("Training tokenized captions saved successfully!")
else:
    print("Error: Training tokenized captions not found.")

file_path = '/content/drive/My Drive/pattern_recognition/lab_2/validation_tokenized_captions.npy'
np.save(file_path, validation_tokenized_captions_as_npy)
if os.path.exists(file_path):
    print("Validation tokenized captions saved successfully!")
else:
    print("Error: Validation tokenized captions not found.")

file_path = '/content/drive/My Drive/pattern_recognition/lab_2/test_tokenized_captions.npy'
np.save(file_path, test_tokenized_captions_as_npy)
if os.path.exists(file_path):
    print("Test tokenized captions saved successfully!")
else:
    print("Error: Test tokenized captions not found.")

file_path = '/content/drive/My Drive/pattern_recognition/lab_2/augmented_tokenized_captions.npy'
np.save(file_path, augmented_tokenized_captions_as_npy)
if os.path.exists(file_path):
    print("Augmented tokenized captions saved successfully!")
else:
    print("Error: Augmented tokenized captions not found.")

file_path = '/content/drive/My Drive/pattern_recognition/lab_2/training_output_labels.npy'
np.save(file_path, training_output_labels_as_npy)
if os.path.exists(file_path):
    print("Training output labels saved successfully!")
else:
    print("Error: Training output labels not found.")

file_path = '/content/drive/My Drive/pattern_recognition/lab_2/validation_output_labels.npy'
np.save(file_path, validation_output_labels_as_npy)
if os.path.exists(file_path):
    print("Validation output labels saved successfully!")
else:
    print("Error: Validation output labels not found.")

file_path = '/content/drive/My Drive/pattern_recognition/lab_2/test_output_labels.npy'
np.save(file_path, test_output_labels_as_npy)
if os.path.exists(file_path):
    print("Test output labels saved successfully!")
else:
    print("Error: Test output labels not found.")

##Sampling images and their captions

In [ ]:
# Training data
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
for i in range(0,10):
  img=train_data['image'][i]
  plt.imshow(img)
  plt.show()
  print(training_captions[i])

In [ ]:
# Augmented training data
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
for i in range(0,10):
  img=augmented_training_images[i]
  plt.imshow(img)
  plt.show()
  print(augmented_training_captions[i])

In [ ]:
# Validation data
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
for i in range(30,50):
  img=validation_data['image'][i]
  plt.imshow(img)
  plt.show()
  print(validation_captions[i])

In [ ]:
# Test data
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
for i in range(0,10):
  img=test_data['image'][i]
  plt.imshow(img)
  # plt.axis('off')  # Turn off axis
  plt.show()
  print(test_captions[i])

##Grouping all validation captions

In [ ]:
grouped_validation_captions = []
for i in range(len(validation_data)):
  grouped_validation_captions.append([])
  grouped_validation_captions[-1].append(validation_data['caption_0'][i])
  grouped_validation_captions[-1].append(validation_data['caption_1'][i])
  grouped_validation_captions[-1].append(validation_data['caption_2'][i])
  grouped_validation_captions[-1].append(validation_data['caption_3'][i])
  grouped_validation_captions[-1].append(validation_data['caption_4'][i])
grouped_validation_captions = np.array(grouped_validation_captions)

In [ ]:
print(grouped_validation_captions[0])

['the boy laying face down on a skateboard is being pushed along the ground by another boy .'
 'Two girls play on a skateboard in a courtyard .'
 'Two people play on a long skateboard .'
 'Two small children in red shirts playing on a skateboard .'
 'two young children on a skateboard going across a sidewalk']


In [ ]:
import os

# Saving extracted features to Google drive to use it later
file_path = '/content/drive/My Drive/pattern_recognition/lab_2/grouped_validation_captions.npy'
np.save(file_path, grouped_validation_captions)
if os.path.exists(file_path):
    print("File saved successfully!")
else:
    print("Error: File not found.")

File saved successfully!


#Step 2: Convolution Neural Network (ResNet)

##Getting feature maps from ResNet

In [ ]:
# Last attribute excludes the top (classification) layers of the model, leaving only the feature extraction layers.
resnet_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False)

# Freezing layers (making them non-trainable)
for layer in resnet_model.layers:
  layer.trainable = False

94765736/94765736 [==============================] - 3s 0us/step


###Passing training data

In [ ]:
# Extracting features from training images
training_images = np.array(train_data['image'])
training_images = np.stack(training_images)
training_images = training_images.astype('float32')
training_images = preprocess_input(training_images)
training_features = resnet_model.predict(training_images)

In [ ]:
import os

# Saving extracted features to Google drive to use it later
file_path = '/content/drive/My Drive/pattern_recognition/lab_2/training_features.npy'
np.save(file_path, training_features)
if os.path.exists(file_path):
    print("File saved successfully!")
else:
    print("Error: File not found.")

In [ ]:
# Accessing features from Google drive if already extracted before
file_path = '/content/drive/My Drive/pattern_recognition/lab_2/training_features.npy'
training_features = np.load(file_path)

In [ ]:
print(training_features.shape)

NameError: name 'training_features' is not defined

###Passing augmented images

In [ ]:
# Extracting features from training images
augmented_training_images = np.array(augmented_training_images)
augmented_training_images= np.stack(augmented_training_images)
augmented_training_images = augmented_training_images.astype('float32')
augmented_training_images = preprocess_input(augmented_training_images)
augmented_training_features = resnet_model.predict(augmented_training_images)

94/94 [==============================] - 596s 6s/step


In [ ]:
import os

# Saving extracted features to Google drive to use it later
file_path = '/content/drive/My Drive/pattern_recognition/lab_2/augmneted_training_features.npy'
np.save(file_path, augmented_training_features)
if os.path.exists(file_path):
    print("File saved successfully!")
else:
    print("Error: File not found.")

File saved successfully!


In [ ]:
# Accessing features from Google drive if already extracted before
file_path = '/content/drive/My Drive/pattern_recognition/lab_2/augmneted_training_features.npy'
augmneted_training_features = np.load(file_path)

In [ ]:
print(augmneted_training_features.shape)

(3000, 7, 7, 2048)


###Passing validation data

In [ ]:
# Extracting features from validation images
validation_images = np.array(validation_data['image'])
validation_images = np.stack(validation_images)
validation_images = validation_images.astype('float32')
validation_images = preprocess_input(validation_images)
validation_features = resnet_model.predict(validation_images)

In [ ]:
import os

# Saving extracted features to Google drive to use it later
file_path = '/content/drive/My Drive/pattern_recognition/lab_2/validation_features.npy'
np.save(file_path, validation_features)
if os.path.exists(file_path):
    print("File saved successfully!")
else:
    print("Error: File not found.")

In [ ]:
# Accessing features from Google drive if already extracted before
file_path = '/content/drive/My Drive/pattern_recognition/lab_2/validation_features.npy'
validation_features = np.load(file_path)

In [ ]:
print(validation_features.shape)

###Passing test data

In [ ]:
# Extracting features from test images
test_images = np.array(test_data['image'])
test_images = np.stack(test_images)
test_images = test_images.astype('float32')
test_images = preprocess_input(test_images)
test_features = resnet_model.predict(test_images)

In [ ]:
import os

# Saving extracted features to Google drive to use it later
file_path = '/content/drive/My Drive/pattern_recognition/lab_2/test_features.npy'
np.save(file_path, test_features)
if os.path.exists(file_path):
    print("File saved successfully!")
else:
    print("Error: File not found.")

In [ ]:
# Accessing features from Google drive if already extracted before
file_path = '/content/drive/My Drive/pattern_recognition/lab_2/test_features.npy'
test_features = np.load(file_path)

In [ ]:
print(test_features.shape)

##Global average pooling

In [ ]:
# Applying global average pooling on ResNet output before passing the maps to the FC layer

global_average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()

training_features = global_average_pooling_layer(training_features)
augmented_training_features = global_average_pooling_layer(augmneted_training_features)
validation_features = global_average_pooling_layer(validation_features)
test_features = global_average_pooling_layer(test_features)

In [ ]:
print(training_features.shape)
print(augmented_training_features.shape)
print(validation_features.shape)

(3000, 2048)


In [ ]:
import os

# Saving training features after pooling to Google drive to use it later
file_path = '/content/drive/My Drive/pattern_recognition/lab_2/training_features_after_pooling.npy'
np.save(file_path, training_features)
if os.path.exists(file_path):
    print("training_features_after_pooling saved successfully!")
else:
    print("Error: training_features_after_pooling not found.")

# Saving augmented training features after pooling to Google drive to use it later
file_path = '/content/drive/My Drive/pattern_recognition/lab_2/augmented_training_features_after_pooling.npy'
np.save(file_path, augmented_training_features)
if os.path.exists(file_path):
    print("augmented_training_features_after_pooling saved successfully!")
else:
    print("Error: augmented_training_features_after_pooling not found.")

# Saving validation features after pooling to Google drive to use it later
file_path = '/content/drive/My Drive/pattern_recognition/lab_2/validation_features_after_pooling.npy'
np.save(file_path, validation_features)
if os.path.exists(file_path):
    print("validation_features_after_pooling saved successfully!")
else:
    print("Error: validation_features_after_pooling not found.")

# Saving test features after pooling to Google drive to use it later
file_path = '/content/drive/My Drive/pattern_recognition/lab_2/test_features_after_pooling.npy'
np.save(file_path, test_features)
if os.path.exists(file_path):
    print("test_features_after_pooling saved successfully!")
else:
    print("Error: test_features_after_pooling not found.")

augmented_training_features_after_pooling saved successfully!


In [ ]:
# getting Training features after pooling from Google drive if already extracted before
file_path = '/content/drive/My Drive/pattern_recognition/lab_2/training_features_after_pooling.npy'
training_features = np.load(file_path)

# getting Validation features after pooling from Google drive if already extracted before
file_path = '/content/drive/My Drive/pattern_recognition/lab_2/validation_features_after_pooling.npy'
validation_features = np.load(file_path)

# getting Test features after pooling from Google drive if already extracted before
file_path = '/content/drive/My Drive/pattern_recognition/lab_2/test_features_after_pooling.npy'
test_features = np.load(file_path)